Shape optimization
==================

## Overview 

In this tutorial, we will optimize a triangle mesh to match a target shape specfied using a set of reference renderings.

Gradients with regards to vertex positions are typically extremely sparse, since only vertices located on visibility discontinuities receive a contribution. As a consequence, naively optimizing a triangle mesh generally results in horrible, tangled meshes.

To avoid this, we will use the method from the paper "[Large Steps in Inverse Rendering of Geometry][1]". This method optimizes a latent variable that enables smoother gradients.

<div class="admonition important alert alert-block alert-success">

🚀 **You will learn how to:**
    
<ul>
  <li>Create a sensor that batches several viewpoints together</li>
  <li>Use the "large steps" algorithm to optimize a shape</li>
  <li>Use remeshing to refine the optimized shape</li>
</ul>
    
</div>

[1]: http://rgl.epfl.ch/publications/Nicolet2021Large

## Setup

As always, let's import `drjit` and `mitsuba` and set a differentiation-aware variant.

In [1]:
import matplotlib.pyplot as plt 
import os

import drjit as dr
import mitsuba as mi
import torch


mi.set_variant('cuda_ad_rgb', 'llvm_ad_rgb')

In [2]:
torch.cuda.set_device(0)
mi.util.dr.set_device(0)

AttributeError: module 'drjit' has no attribute 'set_num_threads'

## Batched rendering

In order to accurately recover a shape, we need several reference renderings, taken from different viewpoints. Rather than rendering each viewpoint separately during the optimisation like in the [volume optimisation tutorial][1], we will use the [<code>batch</code>][2] sensor, that allows us to render all views at once. There is a performance benefit to this approach: the just-in-time compiler only needs to trace the rendering process once instead of multiple times (once for each viewpoint).

Note that we also have to set the `sample_border` flag to `True` for the [<code>hdrfilm</code>][3] plugin. By enabling this option, Mitsuba will render regions slightly beyond the film's boundaries, enabling us to track objects that enter or exit the frame.

[1]: https://mitsuba.readthedocs.io/en/stable/src/inverse_rendering/volume_optimization.html#Optimization
[2]: https://mitsuba.readthedocs.io/en/stable/src/generated/plugins_sensors.html#batch-sensor-batch
[3]: https://mitsuba.readthedocs.io/en/stable/src/generated/plugins_films.html#high-dynamic-range-film-hdrfilm

In [ ]:
sensor_count = 8
sensor = {
    'type': 'batch',
    'film': {
        'type': 'hdrfilm',
        'width': 256 * sensor_count, 'height': 256,
        'filter': {'type': 'gaussian'},
        'sample_border': True
    }
}

The batch sensor expects a list of nested sensors. Here we will generate 8 viewpoints evenly distributed on the sphere, using the [Fibonacci lattice][1].

[1]: http://extremelearning.com.au/evenly-distributing-points-on-a-sphere/

In [ ]:
from mitsuba import ScalarTransform4f as T

golden_ratio = (1 + 5**0.5)/2
for i in range(sensor_count):
    theta = 2 * dr.pi * i / golden_ratio
    phi = dr.acos(1 - 2*(i+0.5)/sensor_count)
    
    d = 5
    origin = [
        d * dr.cos(theta) * dr.sin(phi),
        d * dr.sin(theta) * dr.sin(phi),
        d * dr.cos(phi)
    ]
    
    sensor[f"sensor_{i}"] = {
        'type': 'perspective',
        'fov': 45,
        'to_world': T.look_at(target=[0, 0, 0], origin=origin, up=[0, 1, 0])
    }

## Scene construction

Let's now generate the reference renderings. We will load a scene with the previous sensor, the target mesh, and an environment map. Note the use of the `direct_reparam` integrator, that properly accounts for visibility discontinuities when differentiating, as in the [object pose estimation tutorial][1].

[1]: https://mitsuba.readthedocs.io/en/stable/src/inverse_rendering/object_pose_estimation.html

In [ ]:
scene_dict = {
    'type': 'scene',
    'integrator': {
        'type': 'direct_reparam',
    },
    'sensor': sensor,
    'emitter': {
        'type': 'envmap',
        'filename': "../scenes/textures/envmap2.exr",
    },
    'shape': {
        'type': 'ply',
        'filename': "../scenes/meshes/suzanne.ply",
        'bsdf': {'type': 'diffuse'}
    }
}

scene_target = mi.load_dict(scene_dict)

We can now generate the reference image, our goal is to reconstruct [Blender's Suzanne][1].
When using the [<code>batch</code>][1] sensor, the output is a single image of all viewpoints stitched together horizontally.

[1]: https://en.wikipedia.org/wiki/Blender_(software)
[2]: https://mitsuba.readthedocs.io/en/stable/src/generated/plugins_sensors.html#batch-sensor-batch

In [ ]:
def plot_batch_output(out: mi.TensorXf):
    fig, ax = plt.subplots(figsize=(5*sensor_count, 5))
    ax.imshow(mi.util.convert_to_bitmap(out))
    ax.axis('off');

In [ ]:
ref_img = mi.render(scene_target, spp=256)
plot_batch_output(ref_img)

The starting geometry which we'll optimize is a relatively dense sphere. More challenging scenes and target shapes might require a better initialization.

In [ ]:
scene_dict['shape']['filename'] = '../scenes/meshes/ico_10k.ply'
scene_source = mi.load_dict(scene_dict)

init_img = mi.render(scene_source, spp=128)
plot_batch_output(init_img)

## Naive optimization

It is tempting to apply the same steps as in previous tutorials, i.e. simply render the scene, compute a loss, and differentiate. Let's try that first.

As per usual, all that is needed is an `Optimizer` and a simple optimization loop.

In [ ]:
params = mi.traverse(scene_source)
opt = mi.ad.Adam(lr=1e-1)
opt['shape.vertex_positions'] = params['shape.vertex_positions']

In [ ]:
for it in range(5):
    loss = mi.Float(0.0)

    params.update(opt)

    img = mi.render(scene_source, params, seed=it, spp=16)

    # L1 Loss
    loss = dr.mean(dr.abs(img - ref_img))
    dr.backward(loss)
    opt.step()

    print(f"Iteration {1+it:03d}: Loss = {loss[0]:6f}", end='\r')

In [ ]:
final_img = mi.render(scene_source, spp=128)
plot_batch_output(final_img)

As you can see, even after only a few steps, this approach produces an unusable mess of triangles. This is a consequence of the sparsity of the visibility-related gradients. Indeed, these are only present on edges that are on the silouhette of the mesh for a given viewpoint.

In [ ]:
# Reset the scene
scene_source = mi.load_dict(scene_dict)
params = mi.traverse(scene_source)

## Large Steps Optimization

Rather than directly optimizing cartesian vertex coordinates, the work of [[Nicolet et al. 2021]][1] "_Large Steps in Inverse Rendering of Geometry_" introduces a latent representation that improves the smoothnes of gradients. In short, a latent variable $u$ is defined as $u = (I + \lambda L) v$, where $v$ denotes the vertex positions, $L$ is the (combinatorial) Laplacian of the given mesh, and $\lambda$ is a user-defined hyper-parameter. Intuitively, this parameter $\lambda$ defines by how much the gradients should be smoothed out on the surface.

This approach is readily available in Mitsuba in the `mi.ad.LargeSteps` class. It requires [<code>cholespy</code>][1], a Python package to solve sparse linear systems with Cholesky factorisations.

[1]: http://rgl.epfl.ch/publications/Nicolet2021Large
[2]: https://github.com/rgl-epfl/cholespy

In [ ]:
!pip install cholespy

The `LargeSteps` helper is instanciated from the starting shapes' vertex positions and faces. If the mesh has duplicate vertices (e.g. due to face normals or UV seams), it will internally convert the mesh to a "unique" representation.

In [ ]:
lambda_ = 25
ls = mi.ad.LargeSteps(params['shape.vertex_positions'], params['shape.faces'], lambda_)

We also use a slighlty modified version of the [<code>Adam</code>][1] optimizer, that uses a uniform second moment for all parameters. This can be done by setting `uniform=True` when instantiating the optimizer.

[1]: https://mitsuba.readthedocs.io/en/stable/src/api_reference.html#mitsuba.ad.Adam

In [ ]:
opt = mi.ad.Adam(lr=1e-1, uniform=True)

The `LargeSteps` class has a couple utility methods that can be used to convert back and forth between cartesian and differential representation of the vertex coordinates. The `LargeSteps.to_differential` method is used here to initialize the latent variable.

In [ ]:
opt['u'] = ls.to_differential(params['shape.vertex_positions'])

The optimisation loop must also be slighlty changed. We now need to update the shape using the latent variable, this additional step can be done by using `LargeSteps.from_differential`.

In [ ]:
iterations = 100 if 'PYTEST_CURRENT_TEST' not in os.environ else 5
for it in range(iterations):
    loss = mi.Float(0.0)

    # Retrieve the vertex positions from the latent variable
    params['shape.vertex_positions'] = ls.from_differential(opt['u'])
    params.update()

    img = mi.render(scene_source, params, seed=it, spp=16)

    # L1 Loss
    loss = dr.mean(dr.abs(img - ref_img))
    dr.backward(loss)
    opt.step()

    print(f"Iteration {1+it:03d}: Loss = {loss[0]:6f}", end='\r')

## Intermediate result

In [ ]:
# Update the mesh after the last iteration's gradient step
params['shape.vertex_positions'] = ls.from_differential(opt['u'])
params.update();

Let us again plot the result of our optimization.

In [ ]:
final_img = mi.render(scene_source, spp=128)
plot_batch_output(final_img)

## Remeshing

The result above can be further improved with the help of remeshing. By increasing the tesselation of the mesh, we will be able to recover more details of the target shape. Intuitively, the intent of this step is similar to other "coarse-to-fine" optimization strategies. For example, in the [caustics][1] or the [volume optimization][2] tutorial we increase the resolution of texture that is being optimized over time.

We will use the Botsch-Kobbelt remeshing algorithm provided by the `gpytoolbox` package:

[1]: https://mitsuba.readthedocs.io/en/stable/src/inverse_rendering/caustics_optimization.html
[2]: https://mitsuba.readthedocs.io/en/stable/src/inverse_rendering/volume_optimization.html

In [ ]:
!pip install gpytoolbox

Since the `gpytoolbox` package expects NumPy arrays, we will first convert the mesh data to the correct format.

In [ ]:
import numpy as np
v_np = params['shape.vertex_positions'].numpy().reshape((-1,3)).astype(np.float64)
f_np = params['shape.faces'].numpy().reshape((-1,3))

The Botsch-Kobbelt remeshing algorithm takes a "target edge length" as input argument. This controls the desired tesselation of the mesh. Since we want to increase resolution, we will set this as half of the mean edge length of the current mesh.

In [ ]:
# Compute average edge length
l0 = np.linalg.norm(v_np[f_np[:,0]] - v_np[f_np[:,1]], axis=1)
l1 = np.linalg.norm(v_np[f_np[:,1]] - v_np[f_np[:,2]], axis=1)
l2 = np.linalg.norm(v_np[f_np[:,2]] - v_np[f_np[:,0]], axis=1)

target_l = np.mean([l0, l1, l2]) / 2

We can now run the Botsch-Kobbelt remeshing algorithm. It runs for a user-specified number of iterations, which we set to 5 here. Further details on about this algorithm, can be found it the package's [documentation][1].

[1]: https://gpytoolbox.org/0.1.0/remesh_botsch/

In [ ]:
from gpytoolbox import remesh_botsch

v_new, f_new = remesh_botsch(v_np, f_np, i=5, h=target_l, project=True)

The new vertices and faces must now be passed to our Mitsuba `Mesh`. If the mesh has other attributes (e.g. UV coordinates), they also need to be updated. By default, Mitsuba will reset these to 0 if the vertex or face count is altered.

In [ ]:
params['shape.vertex_positions'] = mi.Float(v_new.flatten().astype(np.float32))
params['shape.faces'] = mi.Int(f_new.flatten())
params.update();

Since the mesh topology has changed, we also need to compute a new latent variable.

In [ ]:
ls = mi.ad.LargeSteps(params['shape.vertex_positions'], params['shape.faces'], lambda_)
opt = mi.ad.Adam(lr=1e-1, uniform=True)
opt['u'] = ls.to_differential(params['shape.vertex_positions'])

Let's continue the optimization.

In [ ]:
iterations = 150 if 'PYTEST_CURRENT_TEST' not in os.environ else 5
for it in range(iterations):
    loss = mi.Float(0.0)

    # Retrieve the vertex positions from the latent variable
    params['shape.vertex_positions'] = ls.from_differential(opt['u'])
    params.update()

    img = mi.render(scene_source, params, seed=it, spp=16)

    # L1 Loss
    loss = dr.mean(dr.abs(img - ref_img))
    dr.backward(loss)
    opt.step()

    print(f"Iteration {1+it:03d}: Loss = {loss[0]:6f}", end='\r')

## Final result

We recover the final state from the latent variable:

In [ ]:
params['shape.vertex_positions'] = ls.from_differential(opt['u'])
params.update();

Finally, let's compare our end result (bottom) to our reference views (top).

In [ ]:
final_img = mi.render(scene_source, spp=128)

fig, ax = plt.subplots(nrows=2, figsize=(5*sensor_count, 10))
ax[0].set_title("Reference", y=0.3, x=-0.005, rotation=90, fontsize=20)
ax[1].set_title("Optimized shape", y=0.2, x=-0.005, rotation=90, fontsize=20)
for i, img in enumerate((ref_img, final_img)):
    ax[i].imshow(mi.util.convert_to_bitmap(img))
    ax[i].axis('off')

Note that the results could be further improved by e.g. using more input views, or using a less agressive step size and more iterations.

## See also

- [<code>mitsuba.ad.LargeSteps</code>][1]
- [<code>direct_reparam</code> plugin][2]
- [<code>batch</code> plugin][3]

[1]: https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.LargeSteps
[2]: https://mitsuba.readthedocs.io/en/latest/src/generated/plugins_integrators.html#reparameterized-direct-integrator-direct-reparam
[3]: https://mitsuba.readthedocs.io/en/latest/src/generated/plugins_sensors.html#batch-sensor-batch